In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("pipeline").getOrCreate()

In [ ]:
# pipeline:
# input -> transformation -> output -> validation -> analysis

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pratyushpuri/crypto-market-sentiment-and-price-dataset-2025")

print("Path to dataset files:", path)

C:\Users\kizer\miniconda3\envs\spark354_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 91.0k/91.0k [00:00<00:00, 534kB/s]

Extracting model files...


Path to dataset files: C:\Users\kizer\.cache\kagglehub\datasets\pratyushpuri\crypto-market-sentiment-and-price-dataset-2025\versions\1


In [2]:
full_path = "C:\\Users\\kizer\\.cache\\kagglehub\\datasets\\pratyushpuri\\crypto-market-sentiment-and-price-dataset-2025\\versions\\1\\crypto_sentiment_prediction_dataset.csv"
# during csv loading we can automatically removes blank lines and remove comments using optional argument 'comment'
# argument 'sep' to point type of separator in file
df = spark.read.csv(full_path, header=True, inferSchema=True)
df.show()

+-------------------+--------------+-----------------+------------------------+------------------+--------------------+----------------------+--------------------+-----------------+---------------------+----------------+----------------+-----------------------+---------------------+
|          timestamp|cryptocurrency|current_price_usd|price_change_24h_percent|trading_volume_24h|      market_cap_usd|social_sentiment_score|news_sentiment_score|news_impact_score|social_mentions_count|fear_greed_index|volatility_index|rsi_technical_indicator|prediction_confidence|
+-------------------+--------------+-----------------+------------------------+------------------+--------------------+----------------------+--------------------+-----------------+---------------------+----------------+----------------+-----------------------+---------------------+
|2025-06-04 20:36:49|      Algorand|           0.3427|                   -5.35|         1716266.1|     1.76212399671E9|                 0.367|      

In [3]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- cryptocurrency: string (nullable = true)
 |-- current_price_usd: double (nullable = true)
 |-- price_change_24h_percent: double (nullable = true)
 |-- trading_volume_24h: double (nullable = true)
 |-- market_cap_usd: double (nullable = true)
 |-- social_sentiment_score: double (nullable = true)
 |-- news_sentiment_score: double (nullable = true)
 |-- news_impact_score: double (nullable = true)
 |-- social_mentions_count: integer (nullable = true)
 |-- fear_greed_index: double (nullable = true)
 |-- volatility_index: double (nullable = true)
 |-- rsi_technical_indicator: double (nullable = true)
 |-- prediction_confidence: double (nullable = true)



In [4]:
social_sentiment_negative_count = df.filter(df['social_sentiment_score'] < 0).count()
print(social_sentiment_negative_count)
print(social_sentiment_negative_count / df.count() * 100)

1003
48.61851672321861


In [ ]:
# validation via joins:
# taking two files, we do right join on file which is valid so only company that match valid company (for example) will be taken
#complex rule validation:
# calculations, against external source, udfs
import pyspark.sql.functions as F
valid_df = df.withColumnRenamed()
joined_df = df.join()
F.broadcast(joined_df)
#need two data frame one on which validation?join could be perform